In [178]:
import zipfile
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
import random


In [179]:
np.random.seed(42)
tf.random.set_seed(42)

In [180]:
zip_file_path = '/content/homer_bart.zip'
extract_dir = 'homer_bart'
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print(os.listdir(extract_dir))


['Bart', 'Homer']


In [181]:
# Step 2: Preprocess the Images
data_dir = 'homer_bart'
batch_size = 32
img_size = (64, 64)
datagen = ImageDataGenerator(rescale=1./255)
image_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)
file_paths = image_generator.filepaths
labels = image_generator.classes


Found 269 images belonging to 2 classes.


In [182]:
# Split into training and test sets with a fixed random seed
train_paths, test_paths, train_labels, test_labels = train_test_split(
    file_paths, labels, test_size=0.1, stratify=labels, random_state=42
)

# Function to load and preprocess images
def load_and_preprocess_image(path):
    img = tf.keras.preprocessing.image.load_img(path, target_size=img_size)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img / 255.0  # Normalize
    return img


In [183]:
# Load and preprocess training and test datasets
train_images = np.array([load_and_preprocess_image(path) for path in train_paths])
test_images = np.array([load_and_preprocess_image(path) for path in test_paths])

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)


In [188]:
# Step 3: Build and Train the Neural Network
model = Sequential([
    Flatten(input_shape=(64, 64, 3)),
    Dense(100, activation='relu'),
    Dense(20, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_dataset, epochs=80)



Epoch 1/80
8/8 [==============================] - 1s 4ms/step - loss: 3.2180 - accuracy: 0.4876
Epoch 2/80
8/8 [==============================] - 0s 4ms/step - loss: 0.8195 - accuracy: 0.5661
Epoch 3/80
8/8 [==============================] - 0s 4ms/step - loss: 0.5830 - accuracy: 0.6777
Epoch 4/80
8/8 [==============================] - 0s 4ms/step - loss: 0.5563 - accuracy: 0.7149
Epoch 5/80
8/8 [==============================] - 0s 4ms/step - loss: 0.5329 - accuracy: 0.7273
Epoch 6/80
8/8 [==============================] - 0s 4ms/step - loss: 0.4960 - accuracy: 0.7479
Epoch 7/80
8/8 [==============================] - 0s 4ms/step - loss: 0.4661 - accuracy: 0.7686
Epoch 8/80
8/8 [==============================] - 0s 4ms/step - loss: 0.4567 - accuracy: 0.7975
Epoch 9/80
8/8 [==============================] - 0s 4ms/step - loss: 0.4705 - accuracy: 0.7686
Epoch 10/80
8/8 [==============================] - 0s 5ms/step - loss: 0.5137 - accuracy: 0.7479
Epoch 11/80
8/8 [======================

In [189]:
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Test Accuracy: {test_accuracy}')


1/1 [==============================] - 0s 183ms/step - loss: 0.4327 - accuracy: 0.8889
Test Accuracy: 0.8888888955116272
